In [11]:
from pathlib import Path
import json
import datetime as dt
from itertools import product
import math
import sys

import googlemaps
import numpy as np
import pandas as pd
import geopandas as gpd

sys.path.append('../')

import googlemaps_helpers as gh

ROOT = Path('../')
DATA_DIR = ROOT/'data'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Initialize the Googlem Maps API client with your API key
#
my_key = gh.get_secret('GOOGLE_KEY', ROOT/'secrets.json')
client = googlemaps.Client(key=my_key)


In [3]:
# Call the Google Maps Distance Matrix API on some sample points
#
path = DATA_DIR/'points.geojson'
points = gpd.read_file(str(path))
print('* shapes\n', points)

origs = [p.coords[0] for p in points['geometry'][:2]]
orig_ids = points[:2].id
dests = [p.coords[0] for p in points['geometry'][2:]]
dest_ids = points[2:].id
time = dt.datetime(2019, 6, 15, 8)
 
r = client.distance_matrix(gh.flip_coords(origs), gh.flip_coords(dests), mode='driving', 
  traffic_model='pessimistic', departure_time=time)
r


* shapes
                                        geometry id
0  POINT (174.7183227539062 -36.78179217131925)  a
1  POINT (174.7567749023438 -36.89829270211866)  b
2  POINT (174.9174499511719 -36.92135192790114)  c


{'destination_addresses': ['26 Ben Nevis Pl, Northpark, Auckland 2013, New Zealand'],
 'origin_addresses': ['9 Waverley Ave, Glenfield, Auckland 0629, New Zealand',
  '919 Mount Eden Rd, Mount Eden, Auckland 1024, New Zealand'],
 'rows': [{'elements': [{'distance': {'text': '31.5 km', 'value': 31495},
     'duration': {'text': '38 mins', 'value': 2304},
     'duration_in_traffic': {'text': '43 mins', 'value': 2564},
     'status': 'OK'}]},
  {'elements': [{'distance': {'text': '18.6 km', 'value': 18597},
     'duration': {'text': '29 mins', 'value': 1739},
     'duration_in_traffic': {'text': '33 mins', 'value': 2000},
     'status': 'OK'}]}],
 'status': 'OK'}

In [9]:
# Build a distance matrix between all pairs of points
# 
n = points.shape[0]
print(gh.cost_build_distance_matrix_df(n**2, num_freebies=2500))

gh.build_distance_matrix_df(client, points, points, origin_id_col='id',
  destination_id_col='id', include_selfies=True)



#elements                                     9
exceeds 10000-element daily limit?        False
estimated cost for job in USD                 0
estimated duration for job in minutes    0.0015
dtype: object


,origin_address,destination_address,origin_id,destination_id,duration,distance
0,"9 Waverley Ave, Glenfield, Auckland 0629, New ...","9 Waverley Ave, Glenfield, Auckland 0629, New ...",a,a,0,0
1,"9 Waverley Ave, Glenfield, Auckland 0629, New ...","919 Mount Eden Rd, Mount Eden, Auckland 1024, ...",a,b,1562,18688
2,"9 Waverley Ave, Glenfield, Auckland 0629, New ...","26 Ben Nevis Pl, Northpark, Auckland 2013, New...",a,c,2304,31495
0,"919 Mount Eden Rd, Mount Eden, Auckland 1024, ...","9 Waverley Ave, Glenfield, Auckland 0629, New ...",b,a,1590,18226
1,"919 Mount Eden Rd, Mount Eden, Auckland 1024, ...","919 Mount Eden Rd, Mount Eden, Auckland 1024, ...",b,b,0,0
2,"919 Mount Eden Rd, Mount Eden, Auckland 1024, ...","26 Ben Nevis Pl, Northpark, Auckland 2013, New...",b,c,1739,18597
0,"26 Ben Nevis Pl, Northpark, Auckland 2013, New...","9 Waverley Ave, Glenfield, Auckland 0629, New ...",c,a,2375,31168
1,"26 Ben Nevis Pl, Northpark, Auckland 2013, New...","919 Mount Eden Rd, Mount Eden, Auckland 1024, ...",c,b,1719,18567
2,"26 Ben Nevis Pl, Northpark, Auckland 2013, New...","26 Ben Nevis Pl, Northpark, Auckland 2013, New...",c,c,0,0
